# Prueba de webscrapping para la obtención de textos de la web

In [ ]:
# Ejemplo con beautiful soap
import requests
from bs4 import BeautifulSoup

URL = "https://www.independent.com/2022/10/01/failed-two-point-conversion-dooms-bishop-diego-in-21-20-loss-to-st-bonaventure/"
page = requests.get(URL)

if page.status_code == 200 and "www.independent.com" in URL:
  print('Bien')
else:
  print('Mal')

soup = BeautifulSoup(page.content, "html.parser")

Bien


In [ ]:
# Titulo
titleSoap = soup.find_all("h1", {"class": "entry-title"})

In [ ]:
for par in results:
  print(par.getText())

Failed Two-Point Conversion Dooms Bishop Diego in 21-20 Loss to St. Bonaventure


In [ ]:
results = soup.find(id="pico")
paragraphs = results.findAll('p')

In [ ]:
import pandas as pd

textos = []

for par in paragraphs:
   textos.append(par.getText())

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(textos, columns=['text'])
df.head()

,text
0,Bishop Diego running back Misa Paiua was stopped short of the goal line on the decisive two-point conversion attempt with 15 seconds to play as St. Bonaventure held on for a 21-20 victory on Friday night at Ventura College.
1,The Marmonte league opener for both teams lived up to the hype of a heavyweight slugfest as both teams relied heavily on the running game to move the football resulting in an extremely physical contest.
2,"In the end it was St. Bonaventure defensive back Zayne McCulley, who made the initial contact on Paiua that decided the game."
3,"“It was one of those things where you start thinking about fatigue factors and who has momentum,” said Bishop Diego coach Tom Crawford of going for the two-point conversion and the win. “I thought we had momentum at that point just the way that drive unfolded. It was kind of unanimous amongst the coaches to go for the two.”"
4,The featured backs for the two teams Delon Thompson and Qu’Ran Gossett engaged in an epic duel as both routinely made spectacular plays with the ball in their hands.


Preprocesado

In [ ]:
# Imports de preprocesamiento de textos
import re
import string

In [ ]:
# Auxiliar function for remove all emoji characters
# https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def removeEmoji(text):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

# Preprocessor function
def preprocessor2(title_text):

    # Primero, segundo y cuarto, eliminiación de enlaces, menciones y hashtags - Incluimos eliminar espacios y caracteres raros
    title_text = title_text.apply(lambda x: re.sub(r'((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b)|(MENTION[0-9]*)|(#[a-zA-Z0-9]*)', ' ', x, flags=re.MULTILINE))
    title_text = title_text.str.replace('\n',' ')
    title_text = title_text.str.replace('\t',' ')

    # Quitamos simbolos de puntuacion
    title_text = title_text.str.translate(str.maketrans('', '', string.punctuation))

    # Eliminamos los restos que no hayan sido eliminados y algunas expresiones que no aportan informacion
    title_text = title_text.apply(lambda x: re.sub(r'([^0-9a-zA-Z:,\s]+)|(rt)|(lol)|(lmao)|(lmfao)', '', x, flags=re.MULTILINE))

    # Despues de eliminar los simbolos de puntuacion, eliminamos los números
    title_text = title_text.apply(lambda x: re.sub(r'([0-9]+)', ' ', x, flags=re.MULTILINE))
    # Tercero, eliminiación de emojis
    title_text = title_text.apply(lambda x: removeEmoji(x))
    # Quitamos palabras sueltas con letras repetidas
    title_text = title_text.apply(lambda x: re.sub(r'(.)\1{3,}', ' ', x, flags=re.MULTILINE))

    # Ponemos todo en minusculas y aplicamos trim
    title_text = title_text.str.lower()
    title_text = title_text.str.strip()
    # Quitamos espacios sobrantes
    title_text = title_text.apply(lambda x: ' '.join([y for y in x.split(' ') if y != '']))

    return title_text

In [ ]:
dfpre = preprocessor2(df['text'])
dfpre.head()

0                                                                                                             bishop diego running back misa paiua was stopped sho of the goal line on the decisive twopoint conversion attempt with seconds to play as st bonaventure held on for a victory on friday night at ventura college
1                                                                                                                     the marmonte league opener for both teams lived up to the hype of a heavyweight slugfest as both teams relied heavily on the running game to move the football resulting in an extremely physical contest
2                                                                                                                                                                                                    in the end it was st bonaventure defensive back zayne mcculley who made the initial contact on paiua that decided the game
3    it was one of those things where yo

## Pruebas con modelo de machine learning

In [ ]:
import pickle

In [ ]:
# Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

df = pd.read_csv('/content/sexism_data_balanced.csv')
df = df.dropna()
vectorizer = TfidfVectorizer(max_features=500, min_df=3, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(df['text'])
X.shape

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


FileNotFoundError: ignored

In [ ]:
X_internet = vectorizer.transform(dfpre)

In [ ]:
# Lectura
with open('/content/bestLGTrain', 'rb') as training_model:
    bestLG = pickle.load(training_model)

In [ ]:
print(bestLG.predict(X_internet))

[0 0 0 0 0 0 0 0 0 0 0]


# MAIN

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

stemmer = WordNetLemmatizer()
def lemmatization(text):
    
    text = text.split()
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

# Auxiliar function for remove all emoji characters
# https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def removeEmoji(text):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

# Preprocessor function
def preprocessor2(title_text):

    # Primero, segundo y cuarto, eliminiación de enlaces, menciones y hashtags - Incluimos eliminar espacios y caracteres raros
    title_text = title_text.apply(lambda x: re.sub(r'((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b)|(MENTION[0-9]*)|(#[a-zA-Z0-9]*)', ' ', x, flags=re.MULTILINE))
    title_text = title_text.str.replace('\n',' ')
    title_text = title_text.str.replace('\t',' ')

    # Quitamos simbolos de puntuacion
    title_text = title_text.str.translate(str.maketrans('', '', string.punctuation))

    # Eliminamos los restos que no hayan sido eliminados y algunas expresiones que no aportan informacion
    title_text = title_text.apply(lambda x: re.sub(r'([^0-9a-zA-Z:,\s]+)|(rt)|(lol)|(lmao)|(lmfao)', '', x, flags=re.MULTILINE))

    # Despues de eliminar los simbolos de puntuacion, eliminamos los números
    title_text = title_text.apply(lambda x: re.sub(r'([0-9]+)', ' ', x, flags=re.MULTILINE))
    # Tercero, eliminiación de emojis
    title_text = title_text.apply(lambda x: removeEmoji(x))
    # Quitamos palabras sueltas con letras repetidas
    title_text = title_text.apply(lambda x: re.sub(r'(.)\1{3,}', ' ', x, flags=re.MULTILINE))

    # Ponemos todo en minusculas y aplicamos trim
    title_text = title_text.str.lower()
    title_text = title_text.str.strip()
    # Quitamos espacios sobrantes
    title_text = title_text.apply(lambda x: ' '.join([y for y in x.split(' ') if y != '']))

    # Lematizacion
    title_text = title_text.apply(lambda x: lemmatization(x))

    return title_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
import requests
from bs4 import BeautifulSoup
# Imports de preprocesamiento de textos
import re
import string
import pickle
import pandas as pd

# Obtenemos la informacion de la web
URL = "https://www.independent.com/2022/09/19/review-maren-morris-brings-a-little-bit-of-country-and-a-whole-lot-of-girl-next-door-charm-to-the-santa-barbara-bowl/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

# Obtenemos los textos que nos interesan
results = soup.find(id="pico")
paragraphs = results.findAll('p')

textos = []
for par in paragraphs:
   textos.append(par.getText())

# Creo el dataframe
df = pd.DataFrame(textos, columns=['text'])
dfpre = preprocessor2(df['text'])

# Lectura de modelos y vectorizer
with open('/content/tfidfVectorizer', 'rb') as vect:
    vectorizer = pickle.load(vect)

with open('/content/bestRFC400Lem', 'rb') as trainded_model:
    model = pickle.load(trainded_model)

X_internet = vectorizer.transform(dfpre)
print(model.predict(X_internet))

[0 0 0 0 0 0]


In [ ]:
df.shape[0]

6

In [ ]:
listSexism = model.predict(X_internet)
listSexism[0] =1

In [ ]:
count = 0
for el in listSexism:
  if el == 1:
    count += 1

count/len(listSexism)

0.16666666666666666

In [ ]:
from textblob.en import subjectivity
# Calculamos la subjetividad y polaridad de cada frase
from textblob import TextBlob

polarity = 0
subjectivity = 0

for index, row in df.iterrows():
    text = row['text']
    blob = TextBlob(text)

    localPol = 0
    localSub = 0
    for sentence in blob.sentences:
      localPol += sentence.sentiment.polarity
      localSub += sentence.sentiment.subjectivity 
      #print('Frase' + str(index) + '. Polaridad: ' + str(sentence.sentiment.polarity) + ' Subjetividad:' + str(sentence.sentiment.subjectivity))

    meanPol = localPol/len(blob.sentences)
    meanSub = localSub/len(blob.sentences)

    polarity += meanPol
    subjectivity += meanSub

# Calculo de la media
polarityMean = polarity/df.shape[0]
subjectivityMean = subjectivity/df.shape[0]
polarityMean,subjectivityMean

(0.16801008597883596, 0.4506666366041366)